<a href="https://colab.research.google.com/github/alexandrufalk/Background-generator/blob/Layout-page/Copy_of_SSD_Single_Shot_MultiBox_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

#Load MobileNetV2
 -pre-trained backbone MobileNetV2 for feature extraction

In [ ]:
backbone=MobileNetV2(input_shape=(300,300,3),include_top=False, weights='imagenet')
# Freeze the backbone layers
backbone.trainable=False

<ipython-input-2-bd1181a1ef32>:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone=MobileNetV2(input_shape=(300,300,3),include_top=False, weights='imagenet')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


#Create SSD Head
-SSD uses convolutional layers for predicting both bounding boxes and object classes.


In [ ]:
def ssd_head(feature_map, num_classes):
  #number of bounding boxes to predict per cell
  num_boxes=4
  #Bounding box regresion
  bbox_regression=tf.keras.layers.Conv2D(num_boxes*4,kernel_size=(3,3),padding='same',activation='linear')(feature_map)

  #Class prediction
  class_prediction=tf.keras.layers.Conv2D(num_boxes*(num_classes+1),kernel_size=(3,3),padding='same')(feature_map)

  return bbox_regression,class_prediction

#Add Multiple Prediction Layers
SSD makes predictions at multiple feature map scales. Add convolutional layers on top of the backbone to predict bounding boxes and classes at different resolutions.

In [ ]:
def build_ssd_model(backbone,num_classes):
  feature_maps=[]
   # Add feature maps from the backbone for different scales
  feature_maps.append(backbone.get_layer('block_13_expand_relu').output)  # High-resolution feature map
  feature_maps.append(backbone.get_layer('block_16_project').output)  # Lower-resolution feature map

  # Create extra feature maps by applying convolutions

  x=backbone.output
  for i in range(3):
    x=tf.keras.layers.Conv2D(256,(3.3),strides=2,padding="same",activation="relu")(x)
    feature_maps.append(x)

  # Collect bounding box and class predictions from each feature map
  bbox_regressions = []
  class_predictions = []

  for feature_map in feature_maps:
    bbox_reg, class_pred=ssd_head(feature_map,num_classes)
    bbox_regressions.append(bbox_reg)
    class_predictions.append(class_pred)

  # Concatenate predictions from different scales
  bbox_regressions=tf.keras.layers.Concatenate(axis=1)(bbox_regressions)
  class_predictions=tf.keras.layers.Concatenate(axis=1)(class_predictions)

  #Define the SSD model
  model=tf.keras.models.Model(inputs=backbone.input,outputs=[bbox_regressions,class_predictions])

  return model


